# Graph Features and API Verification

This notebook validates the following:
1. **API Parity**: Checking if `objects.traverse`, `objects.shortest_path`, etc., are now available.
2. **Graph Features**: Testing `traverse` syntax generation and `RelationDocument` usage.
3. **Shortest Path**: Checking the current status of `shortest_path`.


In [ ]:
import asyncio
from surrealengine import Document, StringField, IntField, RelationDocument, create_connection, ConnectionRegistry

# Setup connection
CONNECTION_URL = "ws://0.0.0.0:8001/rpc"
conn = create_connection(
    url=CONNECTION_URL,
    namespace="test_ns",
    database="test_db",
    username="root",
    password="root",
    make_default=True
)
await conn.connect()
print("Connected!")

## 1. Define Schema

In [ ]:
class Person(Document):
    name = StringField()
    age = IntField()
    
    class Meta:
        collection = "person"

class Knows(RelationDocument):
    since = IntField()
    
    class Meta:
        collection = "knows"

## 2. Populate Data

In [ ]:
async def setup_graph():
    # Clean up
    try:
        await Person.objects.delete()
        await conn.client.query("DELETE knows")
    except Exception as e:
        print(f"Cleanup warning: {e}")
    
    alice = Person(name="Alice", age=30)
    await alice.save()
    
    bob = Person(name="Bob", age=25)
    await bob.save()
    
    charlie = Person(name="Charlie", age=35)
    await charlie.save()
    
    # 1. create_relation (Alice->Bob)
    try:
        relation = await Knows.create_relation(alice, bob, since=2020)
        print(f"Created relation Alice->Bob: {relation.id}")
    except Exception as e:
        print(f"create_relation failed: {e}")
    
    # 2. Manual save (Alice->Charlie)
    # Testing fix: Library should now handle RecordID conversion automatically
    try:
        # Standard usage - no manual casting needed
        rel_manual = Knows(in_document=alice, out_document=charlie, since=2021)
        await rel_manual.save()
        print(f"Created manual relation Alice->Charlie: {rel_manual.id}")
    except Exception as e:
        print(f"Manual save failed: {e}")
        
    return alice, bob, charlie

alice, bob, charlie = await setup_graph()

## 3. Test `traverse`

In [ ]:
qs = Person.objects.traverse("->knows->person")
results = await qs.all()
print(f"Traverse results (raw): {results}")

# Verify Alice's traversed paths
alice_res = [r for r in results if r['id'] == alice.id]
if alice_res:
    traversed = alice_res[0].get('traversed', [])
    print(f"Alice's connections count: {len(traversed)}")
    print(f"Alice's connections: {traversed}")
else:
    print("Rows returned:", len(results))
    pass

## 4. Test `shortest_path`

In [ ]:
try:
    # Find path from Alice to Bob (length 1)
    qs_path = Person.objects.shortest_path(alice.id, bob.id, "knows")
    path = await qs_path.all()
    print("Shortest Path Alice->Bob:", path)
    
    # Find path from Alice to Charlie (length 1)
    qs_path_c = Person.objects.shortest_path(alice.id, charlie.id, "knows")
    path_c = await qs_path_c.all()
    print("Shortest Path Alice->Charlie:", path_c)
    
    # Non-existent path (Bob -> Alice? Directed graph says no unless bidirectional)
    qs_path_none = Person.objects.shortest_path(bob.id, alice.id, "knows")
    path_none = await qs_path_none.all()
    print("Shortest Path Bob->Alice:", path_none)
    
except NotImplementedError as e:
    print(f"Not Implemented: {e}")
except Exception as e:
    print(f"Error: {e}")

## 5. Debug Data Types

In [ ]:
print("\n--- Checking Data Types in DB ---")
try:
    # Verify if 'in' and 'out' are stored as 'record' (links) or something else (e.g. 'string')
    type_debug = await conn.client.query("SELECT id, type::of(in) as in_type, type::of(out) as out_type FROM knows")
    print(type_debug)
except Exception as e:
    print(f"Debug query failed: {e}")